Basket Analysis on an aCommerce category level
==

First thing to do is to connect to the dmbrandcommerce data base

In [2]:
import psycopg2
import pandas as pd
import statsmodels.api as sm
conn=psycopg2.connect(host="dmbrandcommerce.acommerce.platform"
                      ,database="dmbrandcommerce",
                      user="allen", password="8b5uAJjM")

Next is to get raw data, showing all customers and their purchases. We only need the customer ID and SKUs bought. In this case, we test with L'Oreal Thailand with 300k customers and 1300 SKUs. We get the customers with more than 4 unique purchases

In [3]:
from scipy import stats
import numpy as np
query = "WITH list AS (SELECT DISTINCT     COALESCE(shipping_phone,shipping_name) customer,     LOWER(COALESCE(acomm_category_2,acomm_category_1)) AS category  FROM datawarehouse.sat_sales_trx  JOIN datawarehouse.dm_product_attr1 USING(sku,partner_id)  WHERE country = 'TH' AND product_type IN ('ItemMaster','Set')    AND COALESCE(acomm_category_2,acomm_category_1) IS NOT NULL  ), main AS   (  SELECT     customer,     category,     COUNT(category) OVER(PARTITION BY customer) AS row1   FROM list  ) SELECT  customer,   category,   1 AS one FROM main WHERE row1>=5"
df=pd.read_sql_query(query,con=conn)
df

,customer,category,one
0,0123456789,conditioner,1
1,0123456789,facial skincare,1
2,0123456789,hair styling,1
3,0123456789,hair treatment,1
4,0123456789,lipcare,1
...,...,...,...
612075,995768934,baby food & supplement,1
612076,995768934,body skincare,1
612077,995768934,uht milk,1
612078,995768934,body wash,1


Now we pivot the table

In [4]:
basket = df.pivot(index='customer',columns='category',values='one').fillna(0)
basket#.head()

category,adult diaper,aircare,baby diaper,baby food & supplement,blade/razor,body skincare,body wash,breakfast cereal,bundle,cat food,...,sport drink,suncare,talcum,tea,tonic food drink,toothbrush,toothpaste,uht milk,undefined,wipes
customer,,,,,,,,,,,,,,,,,,,,,
+66635935898,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
+66836504593,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0123456789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
021367239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
022151578,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972942245,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
987382829,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
987498649,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now that the data is structured properly, we can generate frequent item sets that have a support of at least 1% (this number was chosen so that I could get enough useful examples). It seems unlikely for a customer to buy more than one product model

In [5]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(basket, min_support=0.07, use_colnames=True)
#frequent_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x)) <=2]

The final step is to generate the rules with their corresponding support, confidence and lift:

In [7]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by = 'confidence', ascending = False)
rules[['antecedents','consequents','confidence']]#[rules['antecedents']=={'detergent'}]

,antecedents,consequents,confidence
368,"(dishwash, fabric softener)",(detergent),0.902962
355,"(conditioner, hair treatment)",(shampoo),0.877419
263,"(dishwash, body wash)",(detergent),0.863990
610,"(body wash, conditioner, facial skincare)",(shampoo),0.862144
61,(dishwash),(detergent),0.857864
...,...,...,...
623,(shampoo),"(body wash, conditioner, facial skincare)",0.116553
562,(shampoo),"(bundle, facial skincare, body skincare)",0.114144
596,(shampoo),"(undefined, facial skincare, body skincare)",0.113077
358,(shampoo),"(conditioner, hair treatment)",0.112337


How to validate this model?